#### Using a Julia 1.7.2 kernel

In [1]:
# import the library
using SL2Cfoam

# Barbero-Immirzi parameter
Immirzi = 1

# initializing sl2cfoam-next   
sl2c_data_folder = "/home/pdona/data_sl2cfoam"
sl2c_config = SL2Cfoam.Config(VerbosityOff, VeryHighAccuracy, 100, 0)
SL2Cfoam.cinit(sl2c_data_folder,Immirzi, sl2c_config)

# truncation parameter
Dl = 10;

In [2]:
function extrapolate(Av_Dl,Av_Dlm1, Av_Dlm2)
    num = Av_Dl * Av_Dlm2 - Av_Dlm1^2 
    den = Av_Dl  - 2* Av_Dlm1 + Av_Dlm2
    num/den
end

extrapolate (generic function with 1 method)

In [3]:
j = 2
j_bulk_min, j_bulk_max = 0, 3j;

In [4]:
# loop over all the possible bulk spins and save the result
for j_bulk = j_bulk_min:j_bulk_max
    spins = [j, j, j, j, j, j, j, j_bulk, j, j]
    v = vertex_compute(spins, Dl-2)
    v = vertex_compute(spins, Dl-1)
    v = vertex_compute(spins, Dl)
end

In [11]:
i_b = 1; i_idx = i_b + 1;
D4_amp_extrapolated = 0.0
D4_amp_pre_extrapolated = 0.0
D4_amp_Dl = 0.0
D4_amp_Dlm1 = 0.0
D4_amp_Dlm2 = 0.0

for j_bulk = j_bulk_min:j_bulk_max
    spins = [j, j, j, j, j, j, j, j_bulk, j, j]
    println(j_bulk)
    AvDl = vertex_compute(spins,Dl).a
    AvDlm1 = vertex_compute(spins,Dl-1).a
    AvDlm2 = vertex_compute(spins,Dl-2).a
    
    AvDlex = extrapolate.(AvDl,AvDlm1,AvDlm2)
    
    D4_partial_amp_Dl = 0.0
    D4_partial_amp_Dlm1 = 0.0
    D4_partial_amp_Dlm2 = 0.0
    D4_partial_amp_pre = 0.0
    
    D = size(AvDl[i_idx,:,:,i_idx,i_idx])[1]
    for i1 in 1:D, i2 in 1:D, i3 in 1:D, i4 in 1:D
        @inbounds D4_partial_amp_Dl += AvDl[i_idx,i2,i1,i_idx,i_idx]*
                                    AvDl[i_idx,i1,i4,i_idx,i_idx]*
                                    AvDl[i_idx,i4,i3,i_idx,i_idx]*
                                    AvDl[i_idx,i3,i2,i_idx,i_idx]     
        @inbounds D4_partial_amp_Dlm1 += AvDlm1[i_idx,i2,i1,i_idx,i_idx]*
                                    AvDlm1[i_idx,i1,i4,i_idx,i_idx]*
                                    AvDlm1[i_idx,i4,i3,i_idx,i_idx]*
                                    AvDlm1[i_idx,i3,i2,i_idx,i_idx]     
        @inbounds D4_partial_amp_Dlm2 += AvDlm2[i_idx,i2,i1,i_idx,i_idx]*
                                    AvDlm2[i_idx,i1,i4,i_idx,i_idx]*
                                    AvDlm2[i_idx,i4,i3,i_idx,i_idx]*
                                    AvDlm2[i_idx,i3,i2,i_idx,i_idx]     
        @inbounds D4_partial_amp_pre += AvDlex[i_idx,i2,i1,i_idx,i_idx]*
                                    AvDlex[i_idx,i1,i4,i_idx,i_idx]*
                                    AvDlex[i_idx,i4,i3,i_idx,i_idx]*
                                    AvDlex[i_idx,i3,i2,i_idx,i_idx]     
    end
    D4_partial_amp_extrapolated = extrapolate(D4_partial_amp_Dl,D4_partial_amp_Dlm1,D4_partial_amp_Dlm2)
    D4_amp_extrapolated += D4_partial_amp_extrapolated  * (2j_bulk + 1)
    
    D4_amp_pre_extrapolated += D4_partial_amp_pre  * (2j_bulk + 1)

    D4_amp_Dl += D4_partial_amp_Dl  * (2j_bulk + 1)
    D4_amp_Dlm1 += D4_partial_amp_Dlm1  * (2j_bulk + 1)
    D4_amp_Dlm2 += D4_partial_amp_Dlm2  * (2j_bulk + 1)
end

0
1
2
3
4
5
6


In [12]:
[D4_amp_extrapolated,extrapolate(D4_amp_Dl,D4_amp_Dlm1,D4_amp_Dlm2),D4_amp_pre_extrapolated]

3-element Vector{Float64}:
 4.015812341964804e-40
 4.015699918071825e-40
 4.008148090441742e-40

In [10]:
extrapolate(D4_amp_Dl,D4_amp_Dlm1,D4_amp_Dlm2)

4.015699918071825e-40